# Building a LangChain Agent with Long-Term Memory via Flotorch

This notebook demonstrates how to build an advanced LangChain agent with long-term memory capabilities using the Flotorch platform. This allows the agent to recall information from previous, separate interactions.

### Key Concepts:
- **Long-Term Memory**: Provided by `FlotorchLangChainMemory` for persistent knowledge across all sessions.
- **`AgentExecutor` and `ChatPromptTemplate`**: Core LangChain components configured to leverage the memory system.

## 1. Setup and Imports

The following cells install the necessary packages, configure API credentials, and import the required components from Flotorch and LangChain, including those needed for external memory.

In [ ]:
# install flotorch langchain package
%pip install  flotorch[langchain]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "https://qa-gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"
USER_ID = "flotorch_user_001"
APP_ID = "flotorch_app_001"
MEMORY_PROVIDER = "<flotorch memory provider>"

In [ ]:
# Import LangChain and related modules
from flotorch.langchain.llm import FlotorchLangChainLLM
from flotorch.langchain.memory import FlotorchLangChainMemory
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool

print("Imported necessary libraries successfully")


## 2. Model Configuration

We initialize the `FlotorchLangChainLLM` which will act as the brain for our agent, enabling it to process information and make decisions.

In [ ]:
model  =  FlotorchLangChainLLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )
print("FlotorchLLM initialized")

## 3. Memory Configuration

This is where we set up the agent's memory system. 

- **Long-Term Memory**: Uses `FlotorchLangChainMemory` to store and retrieve key information across different sessions, acting as the agent's persistent knowledge base.

In [ ]:
longterm_memory = FlotorchLangChainMemory(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id = APP_ID
)

print("short_term_memory initialized")

In [ ]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.
    
    Args:
        text (str): Text to analyze
        
    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""


@tool
def weather(city: str) -> str:
    """Get weather for a city. Available cities: New York, London, Tokyo, Paris."""
    city = city.lower().strip()

    weather_info = {
        "new york": "Sunny, 72°F, Humidity: 45%",
        "london": "Cloudy, 15°C, Humidity: 70%",
        "tokyo": "Rainy, 25°C, Humidity: 80%",
        "paris": "Partly cloudy, 18°C, Humidity: 60%"
    }

    if city in weather_info:
        return f"Weather in {city.title()}: {weather_info[city]}"
    else:
        return f"I don't have weather data for '{city}'. Try: New York, London, Tokyo, or Paris."

tools = [analyze_text, weather]

print("Custom tool defined successfully.")

## 4. Agent and Prompt Configuration

We define the `Agent` logic and `ChatPromptTemplate` with a goal that explicitly instructs it to use its memory capabilities to provide accurate and context-aware responses.

In [ ]:
# Create the Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to tools for calculations, weather, and information search.
    Use the appropriate tool when needed, or answer directly if you can.
    Keep your responses clear and concise.
    
    Memory context: {longterm_history}"""),
    ("human", "{input}"),
    ("placeholder","{agent_scratchpad}"),
])

agent = create_openai_functions_agent(model,tools,prompt)

print(f"Agent is created successfully")

## 5. Agent Executor Assembly with Memory

The `AgentExecutor` is assembled with the agent, tools, and crucially, the `longterm_memory` module. This equips the agent with a persistent memory capability.

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory = longterm_memory,
    verbose=False,
    handle_parsing_errors=True
)


## 6. Interactive Chat

Start a chat session to interact with the agent. Test its ability to recall information you provide from facts it may have learned in previous runs (long-term). Type 'exit' to end.

In [ ]:
while True:
    
    user_query = input("user: ")

    if user_query.lower().strip() == "exit":
        break
    print(f"User: {user_query}")
    response = agent_executor.invoke({"input":user_query})
    print(f"Assistant: {response['output']}")

## Summary

This notebook detailed the implementation of a sophisticated AI agent equipped with long-term memory.  
By leveraging **Flotorch's infrastructure**, we created an agent that retains and recalls information across separate sessions.

### Key Achievements

- **Persistent Memory** 
  Successfully integrated `FlotorchLangChainMemory` for persistent, long-term knowledge.

- **Enhanced Agent Intelligence** 
  The agent demonstrated its ability to recall facts from a knowledge base established in previous interactions, leading to more intelligent and informed responses.

- **Comprehensive Solution** 
  This long-term memory approach provides a powerful template for building advanced agents capable of complex, ongoing, and context-rich dialogues.
